**Tabla de contenido Capitulo 1**

- [Hello Transformers](#Hello-Transformers)
    - [A Tour of Transformer Applications](#A-Tour-of-Transformer-Applications)
    - [Text Classification](#Text-Classification)
    - [Named Entity Recognition](#Named-Entity-Recognition)
    - [Question Answering](#Question-Answering)
    - [Summarization](#Summarization)
    - [Translation](Translation)
    - [Text Generation](#Text-Generation)
- [The Hugging Face Ecosystem](#The-Hugging-Face-Ecosystem)
    - [Hugging Face Tokenizers](#Hugging-Face-Tokenizers)
    - [Hugging Face Datasets](#Hugging-Face-Datasets)
    - [Hugging Face Accelerate](#Hugging-Face-Accelerate)
    - [Main Challenges with Transformers](#Main-Challenges-with-Transformers)

# Hello Transformers

## A Tour of Transformer Applications


Cada tarea de PNL comienza con un fragmento de texto, como los siguientes comentarios inventados de los clientes sobre un determinado sitio en línea orden:

In [1]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

Dependiendo de su aplicación, el texto con el que esté trabajando podría ser un contrato legal, una descripción del producto o algo completamente diferente. En el caso de los comentarios de los clientes, probablemente le gustaría saber si los comentarios son `positivos` o `negativos`.

Esta tarea se denomina `análisis de sentimientos` y forma parte del tema más amplio de clasificación de textos que exploraremos en el capítulo 2. Por ahora, echemos un `vistazo` a lo que `se necesita` para extraer el sentimiento de nuestro texto usando Transformers.


## Text Classification

En Transformers, creamos una instancia de `pipeline` llamando a la función `pipeline()` y proporcionando el nombre de la tarea en la que estamos interesados:

In [3]:
# pip install ipywidgets
from transformers import pipeline
classifier = pipeline("text-classification", framework="pt")  # framework="pt" fuerza PyTorch

2025-06-13 11:26:02.708154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749831962.723179   20001 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749831962.728173   20001 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749831962.740393   20001 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749831962.740408   20001 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749831962.740410   20001 computation_placer.cc:177] computation placer alr

¡Ahora que tenemos nuestro pipeline, generemos algunas predicciones! Cada pipeline toma una cadena de texto (o una lista de cadenas) como entrada y devuelve un lista de predicciones. Cada predicción es un diccionario de Python, por lo que podemos usar Pandas para mostrarlos los resultados.

In [3]:
import pandas as pd
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


En este caso, el modelo confía mucho en que el texto tiene un sentimiento negativo, lo cual tiene sentido dado que estamos lidiando con una queja de un ¡cliente enojado! Tenga en cuenta que para las tareas de análisis de sentimientos, la canalización solo devuelve una de las etiquetas POSITIVAS o NEGATIVAS, ya que la otra puede ser inferido calculando 1-Score.

## Named Entity Recognition

`Predecir el sentimiento de los comentarios de los clientes es un buen primer paso`, pero a menudo desea saber si los comentarios fueron sobre un `artículo o servicio en particular`. En NLP, los objetos del mundo real como productos, lugares y personas se denominan `named entities`, y extraerlos del texto se denomina `named entity recognition(NER)`.
Podemos aplicar NER cargando el pipeline correspondiente y enviándole la reseña de nuestros clientes:

In [5]:
ner_tagger = pipeline("ner", framework="pt",aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556568,Mega,208,212
4,PER,0.590257,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498350,##icons,259,264
7,MISC,0.775361,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


Puede ver que la pipeline detectó todas las entidades y también asignó una categoría como ORG (organización), LOC (ubicación) o PER (persona) a cada uno de ellos. Aquí usamos el argumento `aggregation_strategy` para agrupar las palabras de acuerdo con las predicciones del modelo. Por ejemplo, la entidad `"Optimus Prime"` se compone de dos palabras, pero se le asigna una sola categoría: MISC (miscelánea). `Los puntajes nos dicen cuán confiado estaba el modelo con respecto a las entidades que identificó`. Podemos ver que tenía menos confianza en "Decepticons" y la primera aparición de "Megatron", los cuales no logró agrupar como `single entity`.

Extraer todas las entidades nombradas en un texto es bueno, pero a veces nos gustaría hacer preguntas más específicas. Aquí es donde podemos usar la respuesta a preguntas.

## Question Answering

En `question answering`, proporcionamos al modelo un pasaje de texto llamado `contexto`, junto con una pregunta cuya respuesta nos gustaría extraer. Luego, el modelo devuelve el intervalo de texto correspondiente a la respuesta. Veamos qué obtenemos cuando hacemos una pregunta específica sobre los comentarios de nuestros clientes:

In [6]:
reader = pipeline("question-answering",framework="pt")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


Podemos ver que junto con la respuesta, la pipeline también devolvió números enteros de inicio y finalización que corresponden a los índices de caracteres donde se encuentra la respuesta se encontró (al igual que con el etiquetado NER). Hay varios tipos de respuestas a preguntas que investigaremos en el capítulo 7, pero este tipo en particular se llama `extractive question answering` porque la respuesta se extrae directamente del texto.

## Summarization

`El objetivo del resumen de texto (summarization) es tomar un texto largo como entrada y generar una versión corta con todos los hechos relevantes`. Esta es una tarea mucho más complicada que las anteriores ya que requiere que el modelo genere texto coherente. En lo que debería ser un patrón familiar por ahora, podemos crear una instancia de una pipeline de resumen de la siguiente manera:

In [7]:
summarizer = pipeline("summarization",framework="pt")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0
Your min_length=56 must be inferior than your max_length=45.
/home/luisgarcia/anaconda3/envs/NLP/lib/python3.12/site-packages/transformers/generation/utils.py:1633: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead.


`¡Este resumen no es tan malo!` Aunque se han copiado partes del texto original, el modelo pudo capturar la esencia del problema y identifique correctamente que "Bumblebee" (que aparecía al final) fue el autor de la denuncia. 
En este ejemplo, también puede ver que pasamos algunos argumentos de palabras clave como `max_length` y `clean_up_tokenization_spaces` a la pipeline; estos permiten ajustar las salidas en tiempo de ejecución.

Pero, ¿qué sucede cuando recibes comentarios en un idioma que no entiendes? Puede usar el traductor de Google, o puede usar tu propio transformador para traducirlo por ti.

## Translation

Al igual que el resumen, la traducción es una tarea en la que el resultado consiste en texto generado. Usemos una pipeline de traducción para traducir un texto del Inglés al Alemán:

In [4]:
#pip install sentencepiece
translator = pipeline("translation_en_to_de",framework="pt",model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/home/luisgarcia/anaconda3/envs/NLP/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.


Nuevamente, el modelo produjo una traducción muy buena que usa correctamente los pronombres formales del alemán, como "Ihrem" y " Sie.” 
Aquí también hemos mostrado cómo puede anular el modelo predeterminado en la pipeline para elegir el mejor para su aplicación, y puede encontrar modelos para miles de pares de idiomas en Hugging Face Hub. Antes de dar un paso atrás y analizar todo el Hugging Face Hub, examinemos una última aplicación.

## Text Generation

Digamos que le gustaría poder proporcionar respuestas más rápidas a los comentarios de los clientes al tener acceso a una función de autocompletar. Con un modelo de generación de texto, puede hacer esto de la siguiente manera:

In [5]:
generator = pipeline("text-generation",framework="pt")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. You ordered multiple sets of Megatron figures.

I apologize to the customer for confusion. I realize that you made your mistake and I will not be responsible for it.

If you need a refund of your order, please contact me at bumblebee@gmail.com. I will look into the issue and try to resolve it.

If you already have a copy of your order, please contact me. I will look into it and try to resolv

OK, tal vez no querríamos usar esta finalización para calmar a Bumblebee, pero entiendes la idea general.

Ahora que has visto algunas aplicaciones geniales de los modelos de transformadores, es posible que se pregunte dónde ocurre el entrenamiento. Todos los modelos que hemos utilizado en este capítulo están disponibles públicamente y ya están ajustados para la tarea en cuestión. Sin embargo, en general, querrá ajustar los modelos en sus propios datos, y en los siguientes capítulos aprenderá a hacer precisamente eso.

Pero entrenar un modelo es solo una pequeña parte de cualquier proyecto de NLP: poder procesar datos de manera eficiente, compartir resultados con colegas y hacer que su el trabajo reproducible también es un componente clave. Afortunadamente, Transformers está rodeado por un gran ecosistema de herramientas útiles que admiten gran parte del flujo de trabajo moderno de aprendizaje automático. Echemos un vistazo.

# The Hugging Face Ecosystem

El ecosistema Hugging Face consta principalmente de dos partes: una familia de bibliotecas y el HUB.

Las bibliotecas proporcionan el código, mientras que el HUB proporciona los pesos de los modelo previamente entrenados, los conjuntos de datos, los scripts para las métricas de evaluación y más. En esta sección veremos brevemente los distintos componentes. Vamos a Transformers, como ya lo hemos discutido.

## Hugging Face Tokenizers

Detrás de cada uno de los ejemplos de pipelines que hemos visto en este capítulo hay un paso de tokenización que divide el texto en piezas más pequeñas llamadas tokens. Veremos cómo funciona esto en detalle en el Capítulo 2, pero por ahora es suficiente entender que los `tokens pueden ser palabras, partes de palabras o simplemente caracteres como la puntuación`.

Los modelos de transformadores se entrenan en representaciones numéricas de estos tokens, `así que hacer bien este paso es bastante importante para todo el proyecto de PLN!`

`Tokenizers` ofrece muchas estrategias de tokenización y es extremadamente rápido en la tokenización de texto gracias a su backend en Rust. `También se encarga de todos los pasos de pre y postprocesamiento`, como normalizar las entradas y transformar las salidas del modelo al formato requerido. Con Tokenizers, podemos cargar un tokenizador de la misma manera que podemos cargar pesos de modelos preentrenados con Transformers.

Necesitamos un conjunto de datos y métricas para entrenar y evaluar modelos, así que echemos un vistazo a Datasets, que se encarga de ese aspecto.


## Hugging Face Datasets

Cargar, procesar y almacenar conjuntos de datos puede ser un proceso engorroso, especialmente cuando los conjuntos de datos son demasiado grandes para caber en la RAM de su laptop. Además, generalmente necesita implementar varios scripts para descargar los datos y transformarlos en un formato estándar.

Los conjuntos de datos simplifican este proceso al proporcionar una interfaz estándar para miles de conjuntos de datos que se pueden encontrar en el Hub. También ofrece almacenamiento en caché inteligente (para que no tengas que repetir tu preprocesamiento cada vez que ejecutes tu código) y evita las limitaciones de RAM al aprovechar un mecanismo especial llamado called memory mapping que almacena el contenido de un archivo en la memoria virtual y permite que múltiples procesos modifiquen un archivo de manera más eficiente. La biblioteca también es interoperable con marcos populares como Pandas y NumPy, por lo que no tienes que abandonar la comodidad de tus herramientas favoritas para manejar datos.

¡Con las bibliotecas Transformers, Tokenizers y Datasets tenemos todo lo que necesitamos para entrenar nuestros propios modelos transformer! Sin embargo, como veremos en el Capítulo 10, hay situaciones en las que necesitamos un control detallado sobre el bucle de entrenamiento. Ahí es donde entra en juego la última biblioteca del ecosistema: Accelerate.


## Hugging Face Accelerate

Si alguna vez has tenido que escribir tu propio script de entrenamiento en PyTorch, es probable que hayas tenido algunos dolores de cabeza al intentar trasladar el código que se ejecuta en tu portátil al código que se ejecuta en el clúster de tu organización. Accelerate añade una capa de abstracción a tus bucles de entrenamiento normales que se encarga de toda la lógica personalizada necesaria para la infraestructura de entrenamiento. Esto literalmente acelera tu flujo de trabajo al simplificar el cambio de infraestructura cuando es necesario.

Esto resume los componentes clave del ecosistema de código abierto de Hugging Face. Pero antes de concluir este capítulo, echemos un vistazo a algunos de los desafíos comunes que surgen al intentar implementar transformadores en el mundo real.

## Main Challenges with Transformers

En este capítulo hemos obtenido una visión general de la amplia gama de tareas de PNL que se pueden abordar con modelos de transformadores. Al leer los titulares de los medios, a veces puede parecer que sus capacidades son ilimitadas. Sin embargo, a pesar de su utilidad, `los transformadores están lejos de ser una solución mágica`. Aquí hay algunos desafíos asociados con ellos que exploraremos a lo largo del libro:

- `Lenguaje`: La investigación en PLN está dominada por el idioma inglés. Hay varios modelos para otros idiomas, pero es más difícil encontrar modelos preentrenados para idiomas raros o con pocos recursos. En el Capítulo 4, exploraremos los transformadores multilingües y su capacidad para realizar transferencias cruzadas en cero disparos.
- `Disponibilidad de datos`: Aunque podemos utilizar el aprendizaje por transferencia para reducir drásticamente la cantidad de datos de entrenamiento etiquetados que nuestros modelos necesitan, sigue siendo mucho en comparación con lo que un humano necesita para realizar la tarea. `Abordar escenarios donde tienes pocos o ningún dato etiquetado es el tema del Capítulo 9`.
- `Trabajar con documentos largos`: La autoatención funciona extremadamente bien en textos de párrafo, pero se vuelve muy costosa cuando pasamos a textos más largos como documentos completos. Se discuten enfoques para mitigar esto en el Capítulo 11.
- `Opacidad`: Al igual que otros modelos de aprendizaje profundo, los transformadores son en gran medida opacos. Es difícil o imposible desentrañar "por qué" un modelo hizo una cierta predicción. Este es un desafío especialmente difícil cuando estos modelos se utilizan para tomar decisiones críticas. Exploraremos algunas maneras de investigar los errores de los modelos de transformadores en los Capítulos 2 y 4.
- `Sesgo`: Los modelos de transformadores se preentrenan predominantemente con datos de texto de internet. Esto imprime todos los sesgos que están presentes en los datos en los modelos. Asegurarse de que estos no sean racistas, sexistas es una tarea difícil. Discutimos algunos de estos problemas con más detalle en el Capítulo 10.

Aunque desalentadores, muchos de estos desafíos se pueden superar. Además de en los capítulos específicos mencionados, tocaremos estos temas en casi todos los capítulos que vienen.




# Conclusiones

¡Esperemos que, para este momento, estés emocionado por aprender cómo comenzar a entrenar e integrar estos versátiles modelos en tus propias aplicaciones! Has visto en este capítulo que con solo unas pocas líneas de código puedes utilizar modelos de última generación para clasificación, reconocimiento de entidades nombradas, respuesta a preguntas, traducción y resumen, pero esto es realmente solo la "punta del iceberg."

En los siguientes capítulos aprenderás cómo adaptar transformadores a una amplia gama de casos de uso, como la construcción de un clasificador de texto, o un modelo ligero para producción, o incluso entrenar un modelo de lenguaje desde cero. Adoptaremos un enfoque práctico, lo que significa que para cada concepto que se trate habrá código acompañante que podrás ejecutar en Google Colab o en tu propia máquina con GPU.

Ahora que estamos armados con los conceptos básicos detrás de los transformadores, es hora de ensuciarnos las manos con nuestra primera aplicación: la clasificación de texto. ¡Ese es el tema del próximo capítulo!